# **Support Vector Machines with Kernels - Kernel Machines**

In [1]:
import time
import pickle
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler 
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import get_scorer_names, mean_absolute_percentage_error

from sklearn.svm import SVR

# import helperfunctions
from ML_functions import fun_load_data, fun_preprocessing, fun_fit_tuning
from ML_functions import fun_convert_time
from ML_functions import fun_tuning_results, fun_scores

# Start time count
start_script = time.time()

# Load optimization_problem ('TSP' or 'CVRP') and the size of the train set
with open('settings.pkl', 'rb') as file:
    settings = pickle.load(file)
optimization_problem, train_size_s = settings['optimization_problem'], settings['train_size_s']

# Assign string 'TSP' or 'CVRP' to the following variable to define the optimization problem
optimization_problem = 'TSP'
train_size_s = 0.7

# Load data
data = fun_load_data(optimization_problem)
X, y, train_data = fun_preprocessing(data)

# Create a dictionary to store the results
results_dict = {}

**Support Vector Machine (SVM)**

In [ ]:
# Define a smaller train set due to the long computation times of SVM
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X, y, train_size=0.15, random_state=42)

# Define the model pipeline
model_pipeline = Pipeline(steps=[('scaler', StandardScaler()),
                                 ('model', SVR(kernel='linear', C=1))]) # Regularization parameter C controls trade-off between maximizing the margin and minimizing the classification error (how important it is to satisfy the constraint)

# Estimate model performance with cross-validation on the train set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(model=model_pipeline, X_train=X_train_s, y_train=y_train_s)

**Gaussian Kernel**

In [2]:
# Define a smaller train set due to the long computation times of SVM
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.15, random_state=42)

# Create pipline to scale each fold first during CV
pipe = Pipeline(steps=[('scaler', None), 
                       ('SVM', SVR(kernel='rbf'))])

param_grid = [{'scaler': [StandardScaler()],
              'SVM__C': [300, 400, 500],
              'SVM__gamma': [0.001, 0.01, 0.1]},
              {'scaler': [MinMaxScaler()],
              'SVM__C': [500, 750, 1000],
              'SVM__gamma': [0.01, 0.05, 0.1]}]

grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=3, 
                           scoring='neg_mean_absolute_percentage_error', verbose=True, n_jobs=-1)
fit_time = fun_fit_tuning(search_method=grid_search, X_train=X_train, y_train=y_train, file_name=optimization_problem + '_SVM_GK')

# Estimate model performance with cross validation on the train set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(model=grid_search, X_train=X_train, y_train=y_train)

# View grid search CV scores of all parameter combinations
results_df = fun_tuning_results(search_method=grid_search, search_space=param_grid)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
  CV MAPE train data:  4.5162 %
  CV RMSE train data:  0.7578

  Best model / parameter combination:


{'SVM__C': 750, 'SVM__gamma': 0.05, 'scaler': MinMaxScaler()}

Cross validation scores of different parameter combinations:


,params,mean_test_score
0,"{'SVM__C': 300, 'SVM__gamma': 0.001, 'scaler':...",-0.052999
1,"{'SVM__C': 300, 'SVM__gamma': 0.01, 'scaler': ...",-0.059541
2,"{'SVM__C': 300, 'SVM__gamma': 0.1, 'scaler': S...",-0.522692
3,"{'SVM__C': 400, 'SVM__gamma': 0.001, 'scaler':...",-0.051251
4,"{'SVM__C': 400, 'SVM__gamma': 0.01, 'scaler': ...",-0.059678
5,"{'SVM__C': 400, 'SVM__gamma': 0.1, 'scaler': S...",-0.522692
6,"{'SVM__C': 500, 'SVM__gamma': 0.001, 'scaler':...",-0.049980
7,"{'SVM__C': 500, 'SVM__gamma': 0.01, 'scaler': ...",-0.059727
8,"{'SVM__C': 500, 'SVM__gamma': 0.1, 'scaler': S...",-0.522692
9,"{'SVM__C': 500, 'SVM__gamma': 0.01, 'scaler': ...",-0.071772


In [4]:
# Define a smaller train set due to the long computation times of SVM
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X, y, train_size=0.20, random_state=42)

# Define the model pipeline
pipe = Pipeline(steps=[('scaler', MinMaxScaler()),
                                 ('model', SVR(kernel='rbf', C=750, gamma=0.05))])

# Estimate model performance with cross-validation on the train set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(pipe, X_train_s, y_train_s)

  CV MAPE train data:  4.2866 %
  CV RMSE train data:  1.06
  CV computation time: 4 min, 10 sec

MAPE test data: 3.6768 %
RMSE test data: 0.8985
MAPE and RMSE on test data per instance size:


Number Customers,6,7,8,9,10,11,12,13,14,Mean
MAPE,2.5218,2.7104,3.1515,3.2899,3.6600,4.1348,4.4705,4.5488,5.2411,3.6768
RMSE,0.8278,0.8057,0.8496,0.8897,0.9394,1.0102,1.0518,1.0764,1.1477,0.8985


**Polynomial Kernel**

In [2]:
# Define a smaller train set due to the long computation times of SVM
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.05, random_state=42)

# Create pipline to scale each fold first during CV
pipe = Pipeline(steps=[('scaler', MinMaxScaler()), 
                       ('SVM', SVR(kernel='poly', degree=3))])

param_grid = {'SVM__C': [100, 300, 500],
              'SVM__gamma': [0.05, 0.1, 1]}

grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=3, 
                           scoring='neg_mean_absolute_percentage_error', verbose=True, n_jobs=-1)
fit_time = fun_fit_tuning(search_method=grid_search, X_train=X_train, y_train=y_train, file_name=optimization_problem + '_SVM_PK_GS')

# Estimate model performance with cross validation on the train set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(model=grid_search, X_train=X_train, y_train=y_train)

# View grid search CV scores of all parameter combinations
results_df = fun_tuning_results(search_method=grid_search, search_space=param_grid)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
  CV MAPE train data:  7.0088 %
  CV RMSE train data:  0.6503

  Best model / parameter combination:


{'SVM__C': 300, 'SVM__gamma': 0.1}

Cross validation scores of different parameter combinations:


,params,mean_test_score
0,"{'SVM__C': 100, 'SVM__gamma': 0.05}",-0.099227
1,"{'SVM__C': 100, 'SVM__gamma': 0.1}",-0.074381
2,"{'SVM__C': 100, 'SVM__gamma': 1}",-0.078567
3,"{'SVM__C': 300, 'SVM__gamma': 0.05}",-0.083895
4,"{'SVM__C': 300, 'SVM__gamma': 0.1}",-0.070088
5,"{'SVM__C': 300, 'SVM__gamma': 1}",-0.078567
6,"{'SVM__C': 500, 'SVM__gamma': 0.05}",-0.078231
7,"{'SVM__C': 500, 'SVM__gamma': 0.1}",-0.070676
8,"{'SVM__C': 500, 'SVM__gamma': 1}",-0.078567


In [3]:
# Define a smaller train set due to the long computation times of svm
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X, y, train_size=0.15, random_state=42)

# Define the model pipeline
pipe = Pipeline(steps=[('scaler', MinMaxScaler()),
                                 ('model', SVR(kernel='poly', degree=3, C=300, gamma=0.1))])

# Estimate model performance with cross-validation on the train set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(model=pipe, X_train=X_train_s, y_train=y_train_s)

  CV MAPE train data:  5.2861 %
  CV RMSE train data:  1.3041
  CV computation time: 7 min, 11 sec


# **Compare Results**

In [ ]:
#display(pd.DataFrame(results_dict).sort_values(by='MAPE', axis=1))
print('Total script computation time:', fun_convert_time(start=start_script, end=time.time()))

Total script computation time: 2 min, 6 sec
